In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# For BayesFlow devs: this ensures that the latest dev version can be found
import sys
sys.path.append('../')

import bayesflow as bf
# from bayesflow.distributions.diagonal_normal

In [2]:
def theta_prior():
    theta = np.random.uniform(-1, 1, 2)
    return dict(theta=theta)

def forward_model(theta):
    alpha = np.random.uniform(-np.pi / 2, np.pi / 2)
    r = np.random.normal(0.1, 0.01)
    x1 = -np.abs(theta[0] + theta[1]) / np.sqrt(2) + r * np.cos(alpha) + 0.25
    x2 = (-theta[0] + theta[1]) / np.sqrt(2) + r * np.sin(alpha)
    return dict(x=np.array([x1, x2]))


simulator = bf.make_simulator([theta_prior, forward_model])

# generate 3 random draws from the joint distribution p(r, alpha, theta, x)
sample_data = simulator.sample((3,))

print("Type of sample_data:\n\t", type(sample_data))
print("Keys of sample_data:\n\t", sample_data.keys())
print("Types of sample_data values:\n\t", {k: type(v) for k, v in sample_data.items()})
print("Shapes of sample_data values:\n\t", {k: v.shape for k, v in sample_data.items()})

adapter = (
    bf.adapters.Adapter()
    
    # convert any non-arrays to numpy arrays
    .to_array()
    
    # convert from numpy's default float64 to deep learning friendly float32
    .convert_dtype("float64", "float32")
    
    # standardize target variables to zero mean and unit variance 
    .standardize(exclude="theta")
    
    # rename the variables to match the required approximator inputs
    .rename("theta", "inference_variables")
    .rename("x", "inference_conditions")
)

num_training_batches = 512
num_validation_batches = 128
batch_size = 64
epochs = 30

training_data = simulator.sample(num_training_batches * batch_size,)
validation_data = simulator.sample(num_validation_batches * batch_size,)

flow_matching = bf.networks.FlowMatching(
    subnet="mlp", 
    subnet_kwargs={"widths": (256,)*6 , "dropout": 0.0, "residual": True}
)

flow_matching_workflow = bf.BasicWorkflow(
    simulator=simulator,
    adapter=adapter,
    inference_network=flow_matching,
)

Type of sample_data:
	 <class 'dict'>
Keys of sample_data:
	 dict_keys(['theta', 'x'])
Types of sample_data values:
	 {'theta': <class 'numpy.ndarray'>, 'x': <class 'numpy.ndarray'>}
Shapes of sample_data values:
	 {'theta': (3, 2), 'x': (3, 2)}


In [3]:
print(f"Training data shapes: { {k: v.shape for k, v in training_data.items()} }")
print(f"Validation data shapes: { {k: v.shape for k, v in validation_data.items()} }")


Training data shapes: {'theta': (32768, 2), 'x': (32768, 2)}
Validation data shapes: {'theta': (8192, 2), 'x': (8192, 2)}


In [4]:
history = flow_matching_workflow.fit_offline(
    training_data, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=validation_data
)

INFO:bayesflow:Fitting on dataset instance of OfflineDataset.
INFO:bayesflow:Building on a test batch.


Epoch 1/30
512/512 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2.5293 - loss/inference_loss: 2.5293 - val_loss: 0.4346 - val_loss/inference_loss: 0.4346
Epoch 2/30
451/512 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4026 - loss/inference_loss: 0.4026

KeyboardInterrupt: 

In [13]:
for key, value in training_data.items():
    print(f"{key}: Type={type(value)}, DType={value.dtype if isinstance(value, np.ndarray) else None}")


theta: Type=<class 'numpy.ndarray'>, DType=float64
x: Type=<class 'numpy.ndarray'>, DType=float64
